In [1]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pymupdf

In [2]:
base_model_id = "speakleash/Bielik-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    # llm_int8_enable_fp32_cpu_offload=True,
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
evaluated_ft_model = PeftModel.from_pretrained(base_model, "bielik-legal-finetune/checkpoint-400")

In [4]:
# load eval data
path = 'evaluation_data/umowa wzór.pdf'

all_data = pymupdf.open(path)
all_pages = []
for document in all_data:
    text_page = document.get_textpage().extractText()
    #chunks_per_page = splitter.split_text(text_page)
    all_pages.append(text_page)

print(len(all_pages))
base_context = all_pages[0]

3


In [5]:
def build_prompt(context: str, question: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "Jesteś prawniczym asystentem, ktory precyzyjnie odpowiada na pytania użytkowników",
        },
        {
            "role": "user",
            "content": f"Mając poniższy kontekst {context} odpowiedz proszę na pytanie: {question}"
        }

    ]
    return messages

In [6]:
# query = input("Enter your prompt: ")
#       messages = [
#          {"role": "user", "content": str(query)}
#       ]
#       encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
#       generated_ids = model.generate(encodeds, max_new_tokens=300, do_sample=False)
#       decoded = tokenizer.batch_decode(generated_ids)

In [7]:
eval_questions = [
    "Jakie są strony postępowania?",
    "Jakie są najważniejsze postanowienia niniejszej umowy?",
    "Jakie są najważniejsze daty w poniższej umowie?",
    "Jaki jest adres firmy?",
    "Czy umowa zawiera klauzule abuzywne?",
    "Jaka jest kwota zadatku?",
    #"W jaki sposób przedsiębiorca może udowodnić, że nie znał danego publicznego zapewnienia i rozsądnie nie mógł o nim wiedzieć?"
]

In [17]:
model_input = eval_tokenizer.apply_chat_template(build_prompt(context=base_context, question=eval_questions[4]), return_tensors="pt").to("cuda")
with torch.no_grad():
    generated_ids = evaluated_ft_model.generate(model_input, max_new_tokens=300, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
    #generated_ids_base = base_model.generate(model_input, max_new_tokens=500, do_sample=False, pad_token_id=eval_tokenizer.eos_token_id)
    decoded = eval_tokenizer.batch_decode(generated_ids)
    #decoded_base = eval_tokenizer.batch_decode(generated_ids_base)

# Evaluated model
looked_word = '[/INST]'
inst_end = decoded[0].index(looked_word)
answer = decoded[0][inst_end+len(looked_word):]
print('*' * 20)
print(answer)

# Base model
# inst_end = decoded_base[0].index(looked_word)
# answer_base = decoded_base[0][inst_end+len(looked_word):]
# print('*' * 20)
# print(answer_base)

********************
 Tak, umowa zawiera klauzule abuzywne. Przykładem takiej klauzuli jest zapis dotyczący prawa własności dostarczonych/zamontowanych systemów osłonowych do chwili uregulowania faktury końcowej. Zgodnie z przepisami prawa konsumenckiego, taka klauzula może być uznana za nieważną i niezgodną z zasadami równości stron umowy. </s>


In [ ]:
# how messages look for model
# print(eval_tokenizer.decode(model_input[0]))

In [ ]:
# evaluated_ft_model.eval()
# with torch.no_grad():
#     print(eval_tokenizer.decode(
#         evaluated_ft_model.generate(**model_input, max_new_tokens=600, repetition_penalty=1.15)[0],
#         skip_special_tokens=True))